In [1]:
import glob

import numpy as np
import pandas as pd

from grafting_classifier import GraftingClassifier
from sklearn.linear_model import SGDClassifier
from ogfs_classifier import OGFSClassifier
from osfs_classifier import OSFSClassifier
from dpp_classifier import DPPClassifier
from dpp_classifier_mitra import DPPClassifier as DPPClassifier2
from dpp_classifier_ogfs import DPPClassifier as DPPClassifier3

from sklearn.metrics import log_loss, accuracy_score

#import dask.dataframe as dd
#import dask.array as da

In [2]:
class_train = glob.glob("microarray/*_train.csv")
print(class_train)

['microarray\\colon_train.csv', 'microarray\\leukemia_train.csv', 'microarray\\lung_cancer_train.csv', 'microarray\\prostate_train.csv']


In [3]:
def train_label(fname):
    targetname = fname.replace(".csv", ".labels")
    return pd.read_csv(targetname)

In [4]:
def get_performance(mod, fpath, mod_name):
    train1 = pd.read_csv(fpath).fillna(0)
    y = np.array(train_label(fpath)).flatten()
    
    # simulate streaming...
    # try splitting into groups of ~10,
    # if there is no splits, try ~5.
    train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/10.0) + 1)
    if len(train1_cols) == 1:
        train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/5.0) + 1)
    all_cols = []

    #mod = GraftingClassifier(max_iter=5)
    if mod_name == 'Base':
        mod.fit(train1, y)
        results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
        return results
    
    # lets normalise the dataset...
    train1 = (train1 - train1.mean())/(np.maximum(train1.std(), 1))
    for idx, collist in enumerate(train1_cols):
        if idx == 0:
            column_list = list(np.array(list(train1.columns))[collist])
            mod.fit(train1[column_list], y)
            all_cols.extend(list(collist))
        else:
            all_cols.extend(list(collist))
            column_list = list(np.array(list(train1.columns))[all_cols])
            mod.partial_fit(train1[column_list], y)
        
        # debugging
        print_cond = True if idx % int((len(train1_cols)/10)+1) == 0 else False
        if mod_name in ['Fast_OSFS', 'DPP', 'DPP3', 'OGFS'] and print_cond:
            print("\tmodel: {}, iter: {}".format(mod_name, idx))
        
        # for fast osfs
    if mod_name == 'Fast_OSFS':
        mod._redundancy(train1, y, mode='all')
    
    results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
    return results

In [5]:
def create_models():
    return [
    ('Grafting', GraftingClassifier(max_iter=5, random_state=42)), 
    #('DPP', DPPClassifier(max_iter=5, random_state=42)), 
    #('DPP2', DPPClassifier2(max_iter=5, random_state=42)),
    #('DPP3', DPPClassifier3(max_iter=5, random_state=42)),
    #('OGFS', OGFSClassifier(max_iter=5, random_state=42)),
    #('OSFS', OSFSClassifier(max_iter=5, random_state=42, fast_osfs=False)),
    ('Fast_OSFS', OSFSClassifier(max_iter=5, random_state=42)),
    ('Base', SGDClassifier(loss='log', max_iter=5, random_state=42))
]

In [6]:
ex_dat = class_train[3]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    print(nm, get_performance(mod, ex_dat, mod_name=nm))

microarray\prostate_train.csv (102, 12600)
Grafting {'accuracy': 1.0, 'logloss': 0.0015230494955966597, 'feat_dim': (153,)}
	model: Fast_OSFS, iter: 0
	model: Fast_OSFS, iter: 127
	model: Fast_OSFS, iter: 254
	model: Fast_OSFS, iter: 381
	model: Fast_OSFS, iter: 508
	model: Fast_OSFS, iter: 635
	model: Fast_OSFS, iter: 762
	model: Fast_OSFS, iter: 889
	model: Fast_OSFS, iter: 1016
	model: Fast_OSFS, iter: 1143
		(102, 5537)
		(102, 5536)
		(102, 5535)
		(102, 5534)
		(102, 5533)
		(102, 5532)
		(102, 5531)
		(102, 5530)
		(102, 5529)
		(102, 5528)
		(102, 5527)
		(102, 5526)
		(102, 5525)
		(102, 5524)
		(102, 5523)
		(102, 5522)
		(102, 5521)
		(102, 5520)
		(102, 5519)
		(102, 5518)
		(102, 5517)
		(102, 5516)
		(102, 5515)
		(102, 5514)
		(102, 5513)
		(102, 5512)
		(102, 5511)
		(102, 5510)
		(102, 5509)
		(102, 5508)
		(102, 5507)
		(102, 5506)
		(102, 5505)
		(102, 5504)
		(102, 5503)
		(102, 5502)
		(102, 5501)
		(102, 5500)
		(102, 5499)
		(102, 5498)
		(102, 5497)
		(102, 5496

		(102, 4986)
		(102, 4985)
		(102, 4984)
		(102, 4983)
		(102, 4982)
		(102, 4981)
		(102, 4980)
		(102, 4979)
		(102, 4978)
		(102, 4977)
		(102, 4976)
		(102, 4975)
		(102, 4974)
		(102, 4973)
		(102, 4972)
		(102, 4971)
		(102, 4970)
		(102, 4969)
		(102, 4968)
		(102, 4967)
		(102, 4966)
		(102, 4965)
		(102, 4964)
		(102, 4963)
		(102, 4962)
		(102, 4961)
		(102, 4960)
		(102, 4959)
		(102, 4958)
		(102, 4957)
		(102, 4956)
		(102, 4955)
		(102, 4954)
		(102, 4953)
		(102, 4952)
		(102, 4951)
		(102, 4950)
		(102, 4949)
		(102, 4948)
		(102, 4947)
		(102, 4946)
		(102, 4945)
		(102, 4944)
		(102, 4943)
		(102, 4942)
		(102, 4941)
		(102, 4940)
		(102, 4939)
		(102, 4938)
		(102, 4937)
		(102, 4936)
		(102, 4935)
		(102, 4934)
		(102, 4933)
		(102, 4932)
		(102, 4931)
		(102, 4930)
		(102, 4929)
		(102, 4928)
		(102, 4927)
		(102, 4926)
		(102, 4925)
		(102, 4924)
		(102, 4923)
		(102, 4922)
		(102, 4921)
		(102, 4920)
		(102, 4919)
		(102, 4918)
		(102, 4917)
		(102, 4916)
		(102

		(102, 4405)
		(102, 4404)
		(102, 4403)
		(102, 4402)
		(102, 4401)
		(102, 4400)
		(102, 4399)
		(102, 4398)
		(102, 4397)
		(102, 4396)
		(102, 4395)
		(102, 4394)
		(102, 4393)
		(102, 4392)
		(102, 4391)
		(102, 4390)
		(102, 4389)
		(102, 4388)
		(102, 4387)
		(102, 4386)
		(102, 4385)
		(102, 4384)
		(102, 4383)
		(102, 4382)
		(102, 4381)
		(102, 4380)
		(102, 4379)
		(102, 4378)
		(102, 4377)
		(102, 4376)
		(102, 4375)
		(102, 4374)
		(102, 4374)
		(102, 4373)
		(102, 4372)
		(102, 4371)
		(102, 4370)
		(102, 4369)
		(102, 4368)
		(102, 4367)
		(102, 4366)
		(102, 4365)
		(102, 4364)
		(102, 4363)
		(102, 4362)
		(102, 4361)
		(102, 4360)
		(102, 4359)
		(102, 4358)
		(102, 4357)
		(102, 4356)
		(102, 4355)
		(102, 4354)
		(102, 4353)
		(102, 4352)
		(102, 4351)
		(102, 4350)
		(102, 4349)
		(102, 4348)
		(102, 4347)
		(102, 4346)
		(102, 4345)
		(102, 4344)
		(102, 4343)
		(102, 4342)
		(102, 4341)
		(102, 4340)
		(102, 4339)
		(102, 4338)
		(102, 4337)
		(102, 4336)
		(102

		(102, 3822)
		(102, 3821)
		(102, 3820)
		(102, 3819)
		(102, 3818)
		(102, 3817)
		(102, 3816)
		(102, 3815)
		(102, 3814)
		(102, 3813)
		(102, 3812)
		(102, 3811)
		(102, 3810)
		(102, 3809)
		(102, 3808)
		(102, 3807)
		(102, 3806)
		(102, 3805)
		(102, 3804)
		(102, 3803)
		(102, 3802)
		(102, 3801)
		(102, 3800)
		(102, 3799)
		(102, 3798)
		(102, 3797)
		(102, 3796)
		(102, 3795)
		(102, 3794)
		(102, 3793)
		(102, 3792)
		(102, 3791)
		(102, 3790)
		(102, 3789)
		(102, 3788)
		(102, 3787)
		(102, 3786)
		(102, 3785)
		(102, 3784)
		(102, 3783)
		(102, 3782)
		(102, 3781)
		(102, 3780)
		(102, 3779)
		(102, 3778)
		(102, 3777)
		(102, 3776)
		(102, 3775)
		(102, 3774)
		(102, 3773)
		(102, 3772)
		(102, 3771)
		(102, 3770)
		(102, 3769)
		(102, 3768)
		(102, 3767)
		(102, 3766)
		(102, 3765)
		(102, 3764)
		(102, 3763)
		(102, 3762)
		(102, 3761)
		(102, 3760)
		(102, 3759)
		(102, 3758)
		(102, 3757)
		(102, 3756)
		(102, 3756)
		(102, 3755)
		(102, 3754)
		(102, 3753)
		(102

		(102, 3247)
		(102, 3246)
		(102, 3245)
		(102, 3244)
		(102, 3243)
		(102, 3242)
		(102, 3241)
		(102, 3240)
		(102, 3239)
		(102, 3238)
		(102, 3237)
		(102, 3236)
		(102, 3235)
		(102, 3234)
		(102, 3233)
		(102, 3232)
		(102, 3231)
		(102, 3230)
		(102, 3229)
		(102, 3228)
		(102, 3228)
		(102, 3227)
		(102, 3226)
		(102, 3225)
		(102, 3224)
		(102, 3223)
		(102, 3222)
		(102, 3221)
		(102, 3220)
		(102, 3219)
		(102, 3218)
		(102, 3217)
		(102, 3216)
		(102, 3215)
		(102, 3214)
		(102, 3213)
		(102, 3212)
		(102, 3211)
		(102, 3210)
		(102, 3209)
		(102, 3208)
		(102, 3207)
		(102, 3206)
		(102, 3205)
		(102, 3204)
		(102, 3203)
		(102, 3202)
		(102, 3201)
		(102, 3200)
		(102, 3199)
		(102, 3198)
		(102, 3197)
		(102, 3196)
		(102, 3195)
		(102, 3194)
		(102, 3193)
		(102, 3192)
		(102, 3191)
		(102, 3190)
		(102, 3189)
		(102, 3188)
		(102, 3187)
		(102, 3186)
		(102, 3185)
		(102, 3184)
		(102, 3183)
		(102, 3182)
		(102, 3181)
		(102, 3180)
		(102, 3179)
		(102, 3178)
		(102

...

C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\osfs_util.py:68: RuntimeWarning: invalid value encountered in sqrt
  cor = -V_inv[0][1]/(np.sqrt(V_inv[0][0]*V_inv[1][1]))


		(102, 98)
		(102, 98)
		(102, 98)
		(102, 98)
		(102, 97)


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\kernel_approximation.py:470: UserWarning: n_components > n_samples. This is not possible.
n_components was set to n_samples, which results in inefficient evaluation of the full kernel.
  warnings.warn("n_components > n_samples. This is not possible.\n"


		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
		(102, 97)
Fast_OSFS {'accuracy': 0.41176470588235292, 'logloss': 10.777240152258324, 'feat_dim': (3220,)}
Base {'accuracy': 0.91176470588235292, 'logloss': 3.0475390936685902, 'feat_dim': (12600,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
